In [ ]:
# dependencies import
from common_dependencies import *

from matplotlib import ticker
from matplotlib.patches import Polygon as mplPolygon
from shapely.geometry import Polygon as shPolygon
from shapely.ops import unary_union

from docx.enum.section import WD_ORIENT
from docx import Document
from docx.shared import Inches, Cm
from docx.shared import Pt
from docx.shared import RGBColor

from typing import NamedTuple
import logging
logger = logging.getLogger('main.(02-06)v_make_pred_report.ipynb')

In [ ]:
PATH_TO_GET_DATA = 'data/drawing_data'
PATH_TO_SAVE_DATA = 'data/report'
MODEL_VERS = ['06'] # для каких версий моделей делать отчет
MODEL_NUMS = ['01'] # для каких номеров моделей делать отчет
CALC_LOSS_USING_DEFECT_DEPTH = False

model_runs = r'\d{2}' if len(MODEL_VERS) == 0 else f"({'|'.join([model_ver for model_ver in MODEL_VERS])})"
model_nums = r'\d{2}' if len(MODEL_NUMS) == 0 else f"({'|'.join([model_num for model_num in MODEL_NUMS])})"
# пути к каким версиям и номерам моделей учитывать при создании отчетов
model_folder_regex = r'.*/model_id=' + f'v{model_runs}n{model_nums}$'
run_folder_regex = model_folder_regex[:-1] + r'/run_\d+$'

In [ ]:
def decrease_blur(df_cell_value):
    res = 2*df_cell_value**2 if df_cell_value <= 0.5 else 1-(2*(1-df_cell_value)**2)
    return res

In [ ]:
# create temp folder "report" in 'PATH_TO_DATA' parent folder
# with identical folder tree structure as 'PATH_TO_DATA'
# and fill every folder with jpgs instead of .xlsx files

# BLUR ONES

for run_path in os.walk(PATH_TO_GET_DATA):
    ref = True
    if re.match(run_folder_regex, run_path[0]) is None:
        continue

    full_path_to_save_run_files = run_path[0].replace(PATH_TO_GET_DATA, PATH_TO_SAVE_DATA)
    
    if not os.path.exists(full_path_to_save_run_files):
        os.makedirs(full_path_to_save_run_files)
    
    logger.debug('Current working path: ' + run_path[0])
    for file_name in run_path[2]:
        if re.match('.*\.xlsx', file_name) is None:
            continue
        logger.debug(file_name + ' processing...')

        file_name = file_name.replace('.xlsx','')

        ref_df = pd.read_excel(os.path.join(run_path[0], file_name + '.xlsx'),
             sheet_name='orig_size_reference', index_col=0, dtype=np.float64)
        ref_df = pd.DataFrame(data=dw.normalize_data(ref_df.to_numpy()), 
            index=ref_df.index, columns=ref_df.columns)
        
        pred_df = pd.read_excel(os.path.join(run_path[0], file_name + '.xlsx'),
            sheet_name='orig_size_result', index_col=0, dtype=np.float64)
        pred_df = pd.DataFrame(data=dw.normalize_data(pred_df.to_numpy()), 
            index=pred_df.index, columns=pred_df.columns)

        
        dw.draw_defects_map(ref_df, 
            title = 'REFERENCE ' + re.findall(r"(model_id=v\d{2}n\d{2})", file_name)[0],
            path_to_save = os.path.join(full_path_to_save_run_files, f'REFERENCE {file_name}.jpg'))

        dw.draw_defects_map(pred_df, 
            title = f'PREDICTION {file_name}',
            path_to_save = os.path.join(full_path_to_save_run_files,
                f'PREDICTION {file_name}_loss(use_depth={CALC_LOSS_USING_DEFECT_DEPTH}' +
                f',value={dw.calc_model_prediction_accuracy(pred_df, ref_df, CALC_LOSS_USING_DEFECT_DEPTH):.4f}).jpg'))

        dw.draw_defects_map_with_reference_owerlap(pred_df, ref_df, 
            title = f'OVERLAPPED PREDICTION {file_name}',
            path_to_save = os.path.join(full_path_to_save_run_files,
                f'OVERLAPPED PREDICTION {file_name}_loss(use_depth={CALC_LOSS_USING_DEFECT_DEPTH}' + 
                f',value={dw.calc_model_prediction_accuracy(pred_df, ref_df, CALC_LOSS_USING_DEFECT_DEPTH):.4f}).jpg'))

        
        unblur_pred_df = pred_df.map(decrease_blur)

        dw.draw_defects_map(unblur_pred_df, 
            title = f'UNBLUR PREDICTION {file_name}',
            path_to_save = os.path.join(full_path_to_save_run_files,
                f'UNBLUR PREDICTION {file_name}_loss(use_depth={CALC_LOSS_USING_DEFECT_DEPTH}' + 
                f',value={dw.calc_model_prediction_accuracy(unblur_pred_df, ref_df, CALC_LOSS_USING_DEFECT_DEPTH):.4f}).jpg'))
     
        dw.draw_defects_map_with_reference_owerlap(unblur_pred_df, ref_df, 
            title = 'UNBLUR OVERLAPPED PREDICTION ' + file_name,
            path_to_save = os.path.join(full_path_to_save_run_files,
                f'UNBLUR OVERLAPPED PREDICTION {file_name}_loss(use_depth={CALC_LOSS_USING_DEFECT_DEPTH}' + 
                f',value={dw.calc_model_prediction_accuracy(unblur_pred_df, ref_df, CALC_LOSS_USING_DEFECT_DEPTH):.4f}).jpg'))

In [ ]:
ImageDescription = NamedTuple("ImageDescription", 
                                  [('model_id', str), 
                                   ('run_num', int),
                                   ('image_type_name', str),
                                   ('crop_step', int),
                                   ('path', str),
                                   ('loss', float)])
image_descriptions_list = []


for run_path in os.walk(PATH_TO_SAVE_DATA):
    if re.search(run_folder_regex, run_path[0]) is None:
        continue
    model_id = re.findall(r'model_id=(v\d{2}n\d{2})', run_path[0])[0]
    run_num = re.findall(r'run_(\d+)', run_path[0])[0]
    for image_name in os.listdir(run_path[0]):
        if re.match('.*\.jpg', image_name) is None:
            continue

        crop_step = re.findall(r'step=(\d+)', image_name)[0]
        
        image_type_name = re.search(r'[A-Z\s]+', image_name)[0].strip()
        if not image_type_name == 'REFERENCE':
            loss = re.findall(r'value=(\d+\.\d+)', image_name)[0]
        else:
            loss = ''
        image_descriptions_list.append(ImageDescription(model_id, 
                                                        int(run_num), 
                                                        image_type_name, 
                                                        int(crop_step),
                                                        os.path.join(run_path[0], image_name),
                                                        loss))
        #logger.debug(os.path.join(run_path[0], image_name))

In [ ]:
IMAGE_WIDTH = 8
for model_path in os.walk(PATH_TO_SAVE_DATA):
    if re.match(model_folder_regex, model_path[0]) is None:
        continue
    model_id = re.findall(r'model_id=(v\d{2}n\d{2})', model_path[0])[0]
    # images related to current model
    model_images = [image_desc for image_desc in image_descriptions_list if image_desc.model_id == model_id]
    # numbers of "run_"
    run_numbers = list(set([image_desc.run_num for image_desc in model_images]))
    run_numbers.sort()
    
    document = Document()
    document.add_heading(f'MODEL_ID={model_id} PREDICTION RESULTS', 0)

    for run in run_numbers:
        ref_image = [image_desc for image_desc in model_images 
                     if image_desc.run_num == run and image_desc.image_type_name == 'REFERENCE']

        
        pred_images = [image_desc for image_desc in model_images 
                     if image_desc.run_num == run and image_desc.image_type_name == 'PREDICTION']
        pred_images.sort(key=lambda x: x.crop_step, reverse=True)

        unblur_pred_images = [image_desc for image_desc in model_images 
                 if image_desc.run_num == run and image_desc.image_type_name == 'UNBLUR PREDICTION']
        unblur_pred_images.sort(key=lambda x: x.crop_step, reverse=True)

        
        overlapped_pred_images = [image_desc for image_desc in model_images 
                     if image_desc.run_num == run and image_desc.image_type_name == 'OVERLAPPED PREDICTION']
        overlapped_pred_images.sort(key=lambda x: x.crop_step, reverse=True)

        unblur_overlapped_pred_images = [image_desc for image_desc in model_images 
                     if image_desc.run_num == run and image_desc.image_type_name == 'UNBLUR OVERLAPPED PREDICTION']
        unblur_overlapped_pred_images.sort(key=lambda x: x.crop_step, reverse=True)
        
        document.add_heading(f'RUN {run}', 1)
        document.add_heading('REFERENCE', 2)
        document.add_picture(ref_image[0].path, width=Inches(IMAGE_WIDTH))
                    
        document.add_heading('MODEL PREDICTION', 2)
        for image_desc in pred_images:
            document.add_picture(image_desc.path, width=Inches(IMAGE_WIDTH))

        document.add_heading('UNBLUR MODEL PREDICTION', 2)
        for image_desc in unblur_pred_images:
            document.add_picture(image_desc.path, width=Inches(IMAGE_WIDTH))
        
        document.add_heading('OVERLAPPED WITH REFERENCE MODEL PREDICTION', 2)
        for image_desc in overlapped_pred_images:
            document.add_picture(image_desc.path, width=Inches(IMAGE_WIDTH))

        document.add_heading('UNBLUR OVERLAPPED WITH REFERENCE MODEL PREDICTION', 2)
        for image_desc in unblur_overlapped_pred_images:
            document.add_picture(image_desc.path, width=Inches(IMAGE_WIDTH))
        
        document.add_page_break()
        
    section = document.sections[-1]
    
    #new_width, new_height = section.page_height, section.page_width
    #section.orientation = WD_ORIENT.LANDSCAPE
    #section.page_width = new_width
    #section.page_height = new_height
    
    section.top_margin = Cm(0.5)
    section.bottom_margin = Cm(0.5)
    section.left_margin = Cm(0.5)
    section.right_margin = Cm(0.5)

    head0_style = document.styles['Title']
    head1_style = document.styles['Heading 1']
    head2_style = document.styles['Heading 2']

    head0_style.font.size = Pt(24)
    head1_style.font.size = Pt(22)
    head2_style.font.size = Pt(20)
    
    head0_style.font.color.rgb = RGBColor(255, 0, 0)
    head1_style.font.color.rgb = RGBColor(255, 0, 0)
    head2_style.font.color.rgb = RGBColor(255, 0, 0)
    
    document.save(os.path.join(model_path[0], 'report.docx'))

In [ ]:
IMAGE_WIDTH = 8
for model_path in os.walk(PATH_TO_SAVE_DATA):
    if re.match(model_folder_regex, model_path[0]) is None:
        continue
    model_id = re.findall(r'model_id=(v\d{2}n\d{2})', model_path[0])[0]
    # images related to current model
    model_images = [image_desc for image_desc in image_descriptions_list if image_desc.model_id == model_id]
    # numbers of "run_"
    run_numbers = list(set([image_desc.run_num for image_desc in model_images]))
    run_numbers.sort()
    
    document = Document()
    document.add_heading(f'MODEL_ID={model_id} LOSSES', 0)

    for run in run_numbers:
        ref_image = [image_desc for image_desc in model_images 
                     if image_desc.run_num == run and image_desc.image_type_name == 'REFERENCE']
        
        pred_images = [image_desc for image_desc in model_images 
                     if image_desc.run_num == run and image_desc.image_type_name == 'PREDICTION']
        pred_images.sort(key=lambda x: x.crop_step, reverse=True)

        unblur_pred_images = [image_desc for image_desc in model_images 
                 if image_desc.run_num == run and image_desc.image_type_name == 'UNBLUR PREDICTION']
        unblur_pred_images.sort(key=lambda x: x.crop_step, reverse=True)

        losses = [image_desc.loss for image_desc in pred_images]
        crop_steps = [image_desc.crop_step for image_desc in pred_images]

        unblur_losses = [image_desc.loss for image_desc in unblur_pred_images]
        unblur_crop_steps = [image_desc.crop_step for image_desc in unblur_pred_images]

        fig, ax = plt.subplots()
        fig.set_figwidth(18)
        fig.set_figheight(8)

        plt.plot(crop_steps, losses, 
                 label='Raw prediction loss',  linewidth=1.5, color='blue')
        plt.plot(unblur_crop_steps, unblur_losses,  
                 label='Unblur prediction loss',  linewidth=3, color='red')
        
        ax.set_xlabel('Шаг нарезки кропов', fontsize=20)
        ax.set_ylabel('Значение функции ошибки', fontsize=20)
        ax.set_title(f"График значений функции ошибки от шара нарезки кропов", fontsize=20, pad=15)
        
        ax.patch.set_alpha(0)
        
        #  Устанавливаем форматирование делений:
        ax.tick_params(axis='both', which='both', labelsize = 20)
    
        for loss, crop_size in zip(losses, crop_steps):
            ax.text(crop_size, loss, str(loss), size=15, ha="center")

        for loss, crop_size in zip(unblur_losses, unblur_crop_steps):
            ax.text(crop_size, loss, str(loss), size=15, ha="center")
        
        # Вывод и настройка сетки
        ax.minorticks_on()
        ax.grid(which='major', linewidth=2)
        ax.grid(which='minor', color = 'gray', linestyle = ':')
        ax.legend(fontsize = 20, facecolor = "white", loc = 'upper right')
        
        plt.savefig(os.path.join(model_path[0], f'run_{run} losses graph.jpg'), bbox_inches='tight')
        plt.close()

        document.add_heading(f'RUN {run}', 0)
        document.add_picture(os.path.join(model_path[0], f'run_{run} losses graph.jpg'), width=Inches(IMAGE_WIDTH))
        
    section = document.sections[-1]
    
    #new_width, new_height = section.page_height, section.page_width
    #section.orientation = WD_ORIENT.LANDSCAPE
    #section.page_width = new_width
    #section.page_height = new_height
    
    section.top_margin = Cm(0.5)
    section.bottom_margin = Cm(0.5)
    section.left_margin = Cm(0.5)
    section.right_margin = Cm(0.5)

    head0_style = document.styles['Title']
    head1_style = document.styles['Heading 1']
    head2_style = document.styles['Heading 2']

    head0_style.font.size = Pt(24)
    head1_style.font.size = Pt(22)
    head2_style.font.size = Pt(20)
    
    head0_style.font.color.rgb = RGBColor(255, 0, 0)
    head1_style.font.color.rgb = RGBColor(255, 0, 0)
    head2_style.font.color.rgb = RGBColor(255, 0, 0)
    
    document.save(os.path.join(model_path[0], 'short_report.docx'))